In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
'''OLS'''
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

,Lottery,Literacy,Wealth,Region
0,41,37,73,E
1,38,51,22,N
2,66,13,61,C
3,80,46,76,E
4,79,69,83,E


In [3]:
# categorical variables
res = smf.ols(formula='Lottery ~ Literacy + Wealth + Literacy*Wealth + C(Region)', data=df).fit()
res.summary()
# np.unique(df['Region'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Lottery   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     5.615
Date:                Tue, 04 Jan 2022   Prob (F-statistic):           2.96e-05
Time:                        19:36:48   Log-Likelihood:                -375.30
No. Observations:                  85   AIC:                             766.6
Df Residuals:                      77   BIC:                             786.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          39.0993     17.470      2.238      0.028       4.312      73.887
C(Region)[T.E]    -15.4451      9.807     -1.575      0.119     -34.973       4.082
C(Region)[T.N]     -9.9728      9.432     -1.057      0.294     -28.753       8.808
C(Region)[T.S]     -4.5754      7.380     -0.620      0.537     -19.270      10.119
C(Region)[T.W]    -10.1122      7.275     -1.390      0.169     -24.598       4.374
Literacy           -0.1960      0.396     -0.495      0.622      -0.984       0.592
Wealth              0.4432      0.290      1.530      0.130      -0.133       1.020
Literacy:Wealth     0.0002      0.007      0.031      0.976      -0.013       0.013
==============================================================================
Omnibus:                        3.076   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.215   Jarque-Bera (JB):                2.709
Skew:                          -0.341   Prob(JB):                        0.258
Kurtosis:                       2.452   Cond. No.                     1.56e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
'''binomial'''
star98 = pd.DataFrame(sm.datasets.star98.load_pandas().data)
# no constant
formula = 'SUCCESS ~ -1 + LOWINC + PERASIAN + PERBLACK + PERHISP + PCTCHRT + \
           PCTYRRND + PERMINTE*AVYRSEXP*AVSALK + PERSPENK*PTRATIO*PCTAF'
# constant
formula = 'SUCCESS ~ LOWINC + PERASIAN + PERBLACK + PERHISP + PCTCHRT + \
           PCTYRRND + PERMINTE*AVYRSEXP*AVSALK + PERSPENK*PTRATIO*PCTAF'
dta = star98[['NABOVE', 'NBELOW', 'LOWINC', 'PERASIAN', 'PERBLACK', 'PERHISP',
              'PCTCHRT', 'PCTYRRND', 'PERMINTE', 'AVYRSEXP', 'AVSALK',
              'PERSPENK', 'PTRATIO', 'PCTAF']].copy()
endog = dta['NABOVE'] / (dta['NABOVE'] + dta['NBELOW'])
del dta['NABOVE']
dta['SUCCESS'] = endog

In [5]:
model = smf.glm(formula=formula, data=dta, family=sm.families.Binomial()).fit()
model.summary()
model.params

Intercept                   0.403664
LOWINC                     -0.020396
PERASIAN                    0.015865
PERBLACK                   -0.019802
PERHISP                    -0.009589
PCTCHRT                    -0.002218
PCTYRRND                   -0.002167
PERMINTE                    0.106822
AVYRSEXP                   -0.041119
PERMINTE:AVYRSEXP          -0.003065
AVSALK                      0.013091
PERMINTE:AVSALK            -0.001899
AVYRSEXP:AVSALK             0.000766
PERMINTE:AVYRSEXP:AVSALK    0.000060
PERSPENK                   -0.309703
PTRATIO                     0.009565
PERSPENK:PTRATIO            0.006611
PCTAF                      -0.014274
PERSPENK:PCTAF              0.010513
PTRATIO:PCTAF              -0.000114
PERSPENK:PTRATIO:PCTAF     -0.000246
dtype: float64

In [6]:
'''poisson'''
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [7]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target

In [8]:
formula = "PRICE ~ RM + PTRATIO + LSTAT"
# link = sm.genmod.families.links.log
# family = sm.families.Poisson(link=link)
model = smf.glm(formula=formula, data=df, family=sm.families.Poisson())
result = model.fit() 
result.summary()
# print('AIC:', result.aic)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  PRICE   No. Observations:                  506
Model:                            GLM   Df Residuals:                      502
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1469.4
Date:                Tue, 04 Jan 2022   Deviance:                       468.87
Time:                        19:36:49   Pearson chi2:                     511.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1079      0.157     19.787      0.000       2.800       3.416
RM             0.1540      0.016      9.360      0.000       0.122       0.186
PTRATIO       -0.0321      0.005     -6.913      0.000      -0.041      -0.023
LSTAT         -0.0339      0.002    -17.392      0.000      -0.038      -0.030
==============================================================================
"""